In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import json
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from wordcloud import WordCloud, STOPWORDS

In [ ]:
# For TextRank
import nltk
nltk.download('punkt') # one time execution

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = json.load(open("gdrive/MyDrive/BT4222/Data/politifact_clean.json", "r"))
#gossipcop_tweets = json.load(open("gdrive/MyDrive/BT4222/Data/gossipcop_clean.json", "r"))

In [ ]:
df[0]

{'dataset': 'politifact',
 'id': 'politifact13600',
 'label': 'fake',
 'publish_date': None,
 'publisher': 'enhlive',
 'source': 'https://web.archive.org',
 'text': 'WE WILL RIOT! Michelle Obama’s Mom Will Receive $160k Every Year Out Of Taxpayers’ Pockets!\n\nThis amount of corruption is a next level thing that we haven’t seen portrayed this nonchalantly before. Namely, Michelle Obama’s mom failed to earn a pension that’s three times the amount of an average American family’s whole income!\n\nIt appears to be that just as Barack, the rest of the Obama’s never intended on getting jobs as us, the regular folks, because they got the chance to live on the backs of honest and hard working Americans that pay taxes.\n\nCongress just announced the most stupid, unexplained and unethical waste of taxpayer money prepared by no other than Obama.\n\nFirst Grandma Marian Robinson, 79, will get a big fat check for a lifetime worth 160K government pension when she takes off the White House next year,

In [ ]:
len(df)

954

In [ ]:
long_text_df = list(filter(lambda x: len(x['text'].split())>512, df))
short_text_df = list(filter(lambda x: len(x['text'].split())<513, df))

In [ ]:
len(long_text_df)+len(short_text_df)

954

In [ ]:
long_df = pd.DataFrame(long_text_df)
short_df = pd.DataFrame(short_text_df)

# 1. SUMY-LexRank
- https://iq.opengenus.org/lexrank-text-summarization/
- Cons: contain pronouns while missing their referents when extracted out of context



In [ ]:
!pip install sumy
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

     |████████████████████████████████| 87 kB 4.6 MB/s 
     |████████████████████████████████| 10.1 MB 13.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21712 sha256=23a8cabd376871336e70d3fefc88b5643bd1bcf70f20171c0a31c4699e76ce95
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=634b9d6768871d8b9186a9e6249a8b3ec3f98d3b9e88c272d61c71d815de91fd
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built breadability pycountry


In [ ]:
def summarize(text):    
    SENTENCES_COUNT = 10
    language = 'english'

    parser = PlaintextParser.from_string(text, Tokenizer(language))
    stemmer = Stemmer(language)
    
    summarizer = LexRankSummarizer(stemmer)

    summarizer.stop_words = get_stop_words(language)
    
    summary = []
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        summary.append(str(sentence))

    return " ".join(summary)

In [ ]:
long_df["text"][0]

'Representative Bob Goodlatte will miss out on the fun of issuing a subpoena. Photo: Pete Marovich/Bloomberg via Getty Images\n\nPeter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him. This could lead to dramatic hearings, as Strzok could expose more information about those probes, and is central to President Trump’s effort to discredit Special Counsel Robert Mueller’s investigation.\n\nReports last week indicated that House Judiciary chairman Bob Goodlatte was preparing to subpoena Strzok as part of the House investigation into the FBI’s conduct during the 2016 election. Strzok’s lawyer, Aitan Goelman, said in a letter to Goodlatte released Sunday that the move would be unnecessary, as Strzok “intends to voluntarily appear and testify before 

In [ ]:
len(long_df["text"][0].split())

608

In [ ]:
summarize(long_df["text"][0])

'Photo: Pete Marovich/Bloomberg via Getty Images Peter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him. Reports last week indicated that House Judiciary chairman Bob Goodlatte was preparing to subpoena Strzok as part of the House investigation into the FBI’s conduct during the 2016 election. The letter noted that Strzok cooperated fully with the DOJ inspector general, who issued a report last week that was highly critical of the agent. In one, Page asked if Trump might become president and Strzok replied, “No. No he won’t. Strzok and Page said he meant Trump wouldn’t be elected, and noted that they did not do anything to hurt Trump’s odds, like leaking the existence of the Russia probe before the election. Mueller removed Strzok from his inve

In [ ]:
len(summarize(long_df["text"][0]).split())

236

In [ ]:
# %%time
# df["LexRankSum"] = long_df["text"].apply(summarize)
# df.head()

## ROUGE score
- Returned as a scalar value in the range [0,1] or NaN. A ROUGE score close to zero indicates poor similarity between candidate and references . A ROUGE score close to one indicates strong similarity between candidate and references .

In [ ]:
# !pip install rouge
# from rouge import Rouge

In [ ]:
# rouge = Rouge()
# scores = rouge.get_scores(long_df["Summaries"], long_df["Sumy Summary"])
# df_result = pd.DataFrame(scores)
# df_result.head()

# 2. BART Transformer
- https://huggingface.co/docs/transformers/model_doc/bart
- Use a faster way 

In [ ]:
s = long_df['text'][0]
s

'Representative Bob Goodlatte will miss out on the fun of issuing a subpoena. Photo: Pete Marovich/Bloomberg via Getty Images\n\nPeter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him. This could lead to dramatic hearings, as Strzok could expose more information about those probes, and is central to President Trump’s effort to discredit Special Counsel Robert Mueller’s investigation.\n\nReports last week indicated that House Judiciary chairman Bob Goodlatte was preparing to subpoena Strzok as part of the House investigation into the FBI’s conduct during the 2016 election. Strzok’s lawyer, Aitan Goelman, said in a letter to Goodlatte released Sunday that the move would be unnecessary, as Strzok “intends to voluntarily appear and testify before 

In [ ]:
%%capture
!pip install transformers

In [ ]:
from transformers import pipeline
# Open and read the article
to_tokenize = s

# Initialize the HuggingFace summarization pipeline
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
%%time
summarized = summarizer(to_tokenize, min_length=75, max_length=300)
# Print summarized text
print(summarized)

[{'summary_text': " Peter Strzok, the FBI agent who sent anti-Trump messages to a colleague, has said he's willing to testify before the House Judiciary Committee . House Judiciary chairman Bob Goodlatte was expected to subpoena him . The agent's lawyer said the move would be unnecessary, as he would not invoke his Fifth Amendment rights . President Trump reiterated his thoughts on the matter on Sunday night, calling for Mueller's suspension ."}]
CPU times: user 20.9 s, sys: 107 ms, total: 21 s
Wall time: 25.8 s


[Main Article] <br>
Representative Bob Goodlatte will miss out on the fun of issuing a subpoena. Photo: Pete Marovich/Bloomberg via Getty Images\n\nPeter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him. This could lead to dramatic hearings, as Strzok could expose more information about those probes, and is central to President Trump’s effort to discredit Special Counsel Robert Mueller’s investigation.\n\nReports last week indicated that House Judiciary chairman Bob Goodlatte was preparing to subpoena Strzok as part of the House investigation into the FBI’s conduct during the 2016 election. Strzok’s lawyer, Aitan Goelman, said in a letter to Goodlatte released Sunday that the move would be unnecessary, as Strzok “intends to voluntarily appear and testify before your committee and any other Congressional committee that invites him.”\n\nThe letter noted that Strzok cooperated fully with the DOJ inspector general, who issued a report last week that was highly critical of the agent. In the lead-up to the 2016 election Strzok and former FBI lawyer Lisa Page, who were having an affair, exchanged anti-Trump messages on their FBI-issued phones. In one, Page asked if Trump might become president and Strzok replied, “No. No he won’t. We’ll stop it.” The inspector general found no evidence connecting their views with “specific investigative decisions,” but said that comment and others “implies a willingness to take official action.”\n\nStrzok and Page said he meant Trump wouldn’t be elected, and noted that they did not do anything to hurt Trump’s odds, like leaking the existence of the Russia probe before the election. Goelman told the Washington Post on Sunday that there’s “no question” that Strzok regrets sending the messages, but that “I think what he was doing is expressing his political opinions in what he thought was a private text conversation, and he regrets that this has been weaponized by people with political motivations to try to discredit the Mueller investigation.”\n\nMueller removed Strzok from his investigation several weeks after it started following the discovery of the texts.\n\nGoelman said that if asked to testify, Strzok would be willing to testify without immunity, and would not invoke his Fifth Amendment rights. He said the agent “intends to answer any question put to him, and he intends to defend the integrity of the Clinton email investigation, the Russia collusion investigation to the extent that that’s a topic, and his own integrity.”\n\nTrump reiterated his thoughts on the matter on Sunday night, which are not supported by the inspector general’s findings. The report did not cover the Russia probe, though that did not stop Trump’s allies from calling for Mueller’s immediate suspension.\n\nWhy was the FBI’s sick loser, Peter Strzok, working on the totally discredited Mueller team of 13 Angry & Conflicted Democrats, when Strzok was giving Crooked Hillary a free pass yet telling his lover, lawyer Lisa Page, that “we’ll stop” Trump from becoming President? Witch Hunt! — Donald J. Trump (@realDonaldTrump) June 18, 2018\n\n“The highest level of bias I’ve ever witnessed in any law enforcement officer.” Trey Gowdy on the FBI’s own, Peter Strzok. Also remember that they all worked for Slippery James Comey and that Comey is best friends with Robert Mueller. A really sick deal, isn’t it? — Donald J. Trump (@realDonaldTrump) June 18, 2018\n\nStrzok still works for the FBI, but it’s unclear what he does now. The inspector general referred him, Page, and three other people to the FBI office that handles disciplinary action.

[Summarized Article] <br>
Peter Strzok, the FBI agent who sent anti-Trump messages to a colleague, has said he's willing to testify before the House Judiciary Committee . House Judiciary chairman Bob Goodlatte was expected to subpoena him . The agent's lawyer said the move would be unnecessary, as he would not invoke his Fifth Amendment rights . President Trump reiterated his thoughts on the matter on Sunday night, calling for Mueller's suspension .

# 3. All kind of summary
- gensim
- lexrank
- las
- luhn

https://github.com/miso-belica/sumy


In [ ]:
from gensim.summarization import summarize
from sumy.utils import get_stop_words
from sumy.nlp.tokenizers import Tokenizer as sumytoken
from sumy.nlp.stemmers import Stemmer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.kl import KLSummarizer

In [ ]:
def gensim_summarizer(text):
    return (summarize(text))

def lexrank_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT):
    parser = PlaintextParser.from_string((text), sumytoken(LANGUAGE))
    summarizer_LexRank = LexRankSummarizer(stemmer)
    summarizer_LexRank.stop_words = get_stop_words(LANGUAGE)
    sentences = []
    for sentence in summarizer_LexRank(parser.document, SENTENCES_COUNT):
        a = sentence
        sentences.append(str(a))
    return " ".join(sentences)

def lsa_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT):
    parser = PlaintextParser.from_string((text), sumytoken(LANGUAGE))
    summarizer_lsa = Summarizer(stemmer)
    summarizer_lsa.stop_words = get_stop_words(LANGUAGE)
    sentences = []
    for sentence in summarizer_lsa(parser.document, SENTENCES_COUNT):
        a = sentence
        sentences.append(str(a))
    return " ".join(sentences)

def luhn_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT):
    parser = PlaintextParser.from_string(text, sumytoken(LANGUAGE))
    summarizer_luhn = LuhnSummarizer(stemmer)
    summarizer_luhn.stop_words = get_stop_words(LANGUAGE)
    sentences = []
    for sentence in summarizer_luhn(parser.document, SENTENCES_COUNT):
        a = sentence
        sentences.append(str(a))
    return " ".join(sentences)

def textRank_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT):
    parser = PlaintextParser.from_string(text, sumytoken(LANGUAGE))
    summarizer_textrank = TextRankSummarizer(stemmer)
    summarizer_textrank.stop_words = get_stop_words(LANGUAGE)
    sentences = []
    for sentence in summarizer_textrank(parser.document, SENTENCES_COUNT):
        a = sentence
        sentences.append(str(a))
    return " ".join(sentences)

# https://iq.opengenus.org/edmundson-heuristic-method-for-text-summarization/
def edmundson_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT):
    parser = PlaintextParser.from_string(text, sumytoken(LANGUAGE))
    summarizer_ed = EdmundsonSummarizer(stemmer)
    summarizer_ed.stop_words = get_stop_words(LANGUAGE)
    summarizer_ed.bonus_words = ("fake")# Enter list of bonus words here
    summarizer_ed.stigma_words = ("fake")# Enter list of stigma words here
    summarizer_ed.null_words = ("fake")# Enter list of null words here
    sentences = []
    for sentence in summarizer_ed(parser.document, SENTENCES_COUNT):
        a = sentence
        sentences.append(str(a))
    return " ".join(sentences)

def kl_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT):
    parser = PlaintextParser.from_string(text, sumytoken(LANGUAGE))
    summarizer_kl = KLSummarizer(stemmer)
    summarizer_kl.stop_words = get_stop_words(LANGUAGE)
    sentences = []
    for sentence in summarizer_kl(parser.document, SENTENCES_COUNT):
        a = sentence
        sentences.append(str(a))
    return " ".join(sentences)

In [ ]:
len(long_df['text'][0].split())

608

In [ ]:
long_df['text'][0]

'Representative Bob Goodlatte will miss out on the fun of issuing a subpoena. Photo: Pete Marovich/Bloomberg via Getty Images\n\nPeter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him. This could lead to dramatic hearings, as Strzok could expose more information about those probes, and is central to President Trump’s effort to discredit Special Counsel Robert Mueller’s investigation.\n\nReports last week indicated that House Judiciary chairman Bob Goodlatte was preparing to subpoena Strzok as part of the House investigation into the FBI’s conduct during the 2016 election. Strzok’s lawyer, Aitan Goelman, said in a letter to Goodlatte released Sunday that the move would be unnecessary, as Strzok “intends to voluntarily appear and testify before 

In [ ]:
LANGUAGE = "english"
SENTENCES_COUNT = 15 # dk how to collaborate this
stemmer = Stemmer(LANGUAGE)
text = long_df['text'][0]

gensim_summary = gensim_summarizer(text)
lexrank_summary = lexrank_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT)
lsa_summary = lsa_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT)
luhn_summary = luhn_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT)
textrank_summary = textRank_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT)
edmundson_summary = edmundson_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT)
kl_summary = kl_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT)

[Main Article] <br>
Representative Bob Goodlatte will miss out on the fun of issuing a subpoena. Photo: Pete Marovich/Bloomberg via Getty Images\n\nPeter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him. This could lead to dramatic hearings, as Strzok could expose more information about those probes, and is central to President Trump’s effort to discredit Special Counsel Robert Mueller’s investigation.\n\nReports last week indicated that House Judiciary chairman Bob Goodlatte was preparing to subpoena Strzok as part of the House investigation into the FBI’s conduct during the 2016 election. Strzok’s lawyer, Aitan Goelman, said in a letter to Goodlatte released Sunday that the move would be unnecessary, as Strzok “intends to voluntarily appear and testify before your committee and any other Congressional committee that invites him.”\n\nThe letter noted that Strzok cooperated fully with the DOJ inspector general, who issued a report last week that was highly critical of the agent. In the lead-up to the 2016 election Strzok and former FBI lawyer Lisa Page, who were having an affair, exchanged anti-Trump messages on their FBI-issued phones. In one, Page asked if Trump might become president and Strzok replied, “No. No he won’t. We’ll stop it.” The inspector general found no evidence connecting their views with “specific investigative decisions,” but said that comment and others “implies a willingness to take official action.”\n\nStrzok and Page said he meant Trump wouldn’t be elected, and noted that they did not do anything to hurt Trump’s odds, like leaking the existence of the Russia probe before the election. Goelman told the Washington Post on Sunday that there’s “no question” that Strzok regrets sending the messages, but that “I think what he was doing is expressing his political opinions in what he thought was a private text conversation, and he regrets that this has been weaponized by people with political motivations to try to discredit the Mueller investigation.”\n\nMueller removed Strzok from his investigation several weeks after it started following the discovery of the texts.\n\nGoelman said that if asked to testify, Strzok would be willing to testify without immunity, and would not invoke his Fifth Amendment rights. He said the agent “intends to answer any question put to him, and he intends to defend the integrity of the Clinton email investigation, the Russia collusion investigation to the extent that that’s a topic, and his own integrity.”\n\nTrump reiterated his thoughts on the matter on Sunday night, which are not supported by the inspector general’s findings. The report did not cover the Russia probe, though that did not stop Trump’s allies from calling for Mueller’s immediate suspension.\n\nWhy was the FBI’s sick loser, Peter Strzok, working on the totally discredited Mueller team of 13 Angry & Conflicted Democrats, when Strzok was giving Crooked Hillary a free pass yet telling his lover, lawyer Lisa Page, that “we’ll stop” Trump from becoming President? Witch Hunt! — Donald J. Trump (@realDonaldTrump) June 18, 2018\n\n“The highest level of bias I’ve ever witnessed in any law enforcement officer.” Trey Gowdy on the FBI’s own, Peter Strzok. Also remember that they all worked for Slippery James Comey and that Comey is best friends with Robert Mueller. A really sick deal, isn’t it? — Donald J. Trump (@realDonaldTrump) June 18, 2018\n\nStrzok still works for the FBI, but it’s unclear what he does now. The inspector general referred him, Page, and three other people to the FBI office that handles disciplinary action.

In [ ]:
len(gensim_summary.split())

210

In [ ]:
gensim_summary

'Peter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him.\nIn the lead-up to the 2016 election Strzok and former FBI lawyer Lisa Page, who were having an affair, exchanged anti-Trump messages on their FBI-issued phones.\nStrzok and Page said he meant Trump wouldn’t be elected, and noted that they did not do anything to hurt Trump’s odds, like leaking the existence of the Russia probe before the election.\nGoelman told the Washington Post on Sunday that there’s “no question” that Strzok regrets sending the messages, but that “I think what he was doing is expressing his political opinions in what he thought was a private text conversation, and he regrets that this has been weaponized by people with political motivations to try to discredit the Mu

In [ ]:
len(lexrank_summary.split())

281

In [ ]:
lexrank_summary

'Photo: Pete Marovich/Bloomberg via Getty Images Peter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him. Reports last week indicated that House Judiciary chairman Bob Goodlatte was preparing to subpoena Strzok as part of the House investigation into the FBI’s conduct during the 2016 election. The letter noted that Strzok cooperated fully with the DOJ inspector general, who issued a report last week that was highly critical of the agent. In one, Page asked if Trump might become president and Strzok replied, “No. No he won’t. Strzok and Page said he meant Trump wouldn’t be elected, and noted that they did not do anything to hurt Trump’s odds, like leaking the existence of the Russia probe before the election. Mueller removed Strzok from his inve

In [ ]:
len(lsa_summary.split())

445

In [ ]:
lsa_summary

'Representative Bob Goodlatte will miss out on the fun of issuing a subpoena. Peter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him. This could lead to dramatic hearings, as Strzok could expose more information about those probes, and is central to President Trump’s effort to discredit Special Counsel Robert Mueller’s investigation. Strzok’s lawyer, Aitan Goelman, said in a letter to Goodlatte released Sunday that the move would be unnecessary, as Strzok “intends to voluntarily appear and testify before your committee and any other Congressional committee that invites him.” The letter noted that Strzok cooperated fully with the DOJ inspector general, who issued a report last week that was highly critical of the agent. In the lead-up to the 20

In [ ]:
len(textrank_summary.split())

410

In [ ]:
textrank_summary

'Photo: Pete Marovich/Bloomberg via Getty Images Peter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him. This could lead to dramatic hearings, as Strzok could expose more information about those probes, and is central to President Trump’s effort to discredit Special Counsel Robert Mueller’s investigation. Reports last week indicated that House Judiciary chairman Bob Goodlatte was preparing to subpoena Strzok as part of the House investigation into the FBI’s conduct during the 2016 election. Strzok’s lawyer, Aitan Goelman, said in a letter to Goodlatte released Sunday that the move would be unnecessary, as Strzok “intends to voluntarily appear and testify before your committee and any other Congressional committee that invites him.” The letter 

In [ ]:
len(edmundson_summary.split())

418

In [ ]:
edmundson_summary

'Representative Bob Goodlatte will miss out on the fun of issuing a subpoena. Photo: Pete Marovich/Bloomberg via Getty Images Peter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him. This could lead to dramatic hearings, as Strzok could expose more information about those probes, and is central to President Trump’s effort to discredit Special Counsel Robert Mueller’s investigation. Reports last week indicated that House Judiciary chairman Bob Goodlatte was preparing to subpoena Strzok as part of the House investigation into the FBI’s conduct during the 2016 election. Strzok’s lawyer, Aitan Goelman, said in a letter to Goodlatte released Sunday that the move would be unnecessary, as Strzok “intends to voluntarily appear and testify before your c

In [ ]:
len(kl_summary.split())

260

In [ ]:
kl_summary

'Representative Bob Goodlatte will miss out on the fun of issuing a subpoena. Photo: Pete Marovich/Bloomberg via Getty Images The letter noted that Strzok cooperated fully with the DOJ inspector general, who issued a report last week that was highly critical of the agent. No he won’t. We’ll stop it.” The inspector general found no evidence connecting their views with “specific investigative decisions,” but said that comment and others “implies a willingness to take official action.” He said the agent “intends to answer any question put to him, and he intends to defend the integrity of the Clinton email investigation, the Russia collusion investigation to the extent that that’s a topic, and his own integrity.” Trump reiterated his thoughts on the matter on Sunday night, which are not supported by the inspector general’s findings. The report did not cover the Russia probe, though that did not stop Trump’s allies from calling for Mueller’s immediate suspension. Why was the FBI’s sick lose

In [ ]:
len(luhn_summary.split())

425

In [ ]:
luhn_summary

'Representative Bob Goodlatte will miss out on the fun of issuing a subpoena. Peter Strzok, the FBI agent who sent anti-Trump messages to a colleague while leading the investigation of Hillary Clinton’s emails and Russia’s involvement in the Trump campaign, has said he’s willing to testify before the House Judiciary Committee, or any other committee that wants to talk with him. This could lead to dramatic hearings, as Strzok could expose more information about those probes, and is central to President Trump’s effort to discredit Special Counsel Robert Mueller’s investigation. Reports last week indicated that House Judiciary chairman Bob Goodlatte was preparing to subpoena Strzok as part of the House investigation into the FBI’s conduct during the 2016 election. Strzok’s lawyer, Aitan Goelman, said in a letter to Goodlatte released Sunday that the move would be unnecessary, as Strzok “intends to voluntarily appear and testify before your committee and any other Congressional committee t